<a href="https://colab.research.google.com/github/Seng-Pan/llm_project/blob/main/QnA_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [ ]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65


In [ ]:
!pip install langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


In [ ]:
!pip install -U langgraph langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.6/364.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.45
    Uninstalling langsmith-0.3.45:
      Successfully uninstalled langsmith-0.3.45


In [ ]:
!pip install -U "langchain[openai]"

### Main Chatbot

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, ValidationError
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from langchain.prompts import PromptTemplate
from typing import Annotated, Optional, List
from IPython.display import Image, display
from langgraph.prebuilt import ToolNode
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
import getpass
import os


# GLOBAL constants
OPENAI_KEY="your api key"
API_KEY="OPENAI_API_KEY"
CHAT_MODEL="gpt-4o-mini"
NODE1="chatbot"

# ==== API KEY SETUP ====
os.environ[API_KEY] = OPENAI_KEY

# ==== Initialize Chat Model ====
llm = ChatOpenAI(model=CHAT_MODEL, openai_api_key=os.environ[API_KEY])

class State(TypedDict):
    messages: Annotated[list, add_messages]
    collected_data: list
    is_finished: bool

graph_builder = StateGraph(State)


# ==== Prompt Templates ====
COLLECTION_TEMPLATE = PromptTemplate.from_template("""
You are a warm and helpful assistant that guides users step-by-step through applying for a small business loan (20–50 USD).
Use a friendly tone, short messages, and always explain why a question is being asked.
Adjust questions dynamically based on the user's situation (e.g., if they don’t have a business yet).
Let users skip optional uploads or clarifications when needed.

🎯 Goal:
Collect key details for credit evaluation and return a structured JSON of user responses.

🔄 Conversation Style:
- Do **not** say “Thank you”, “Great!”, or “Thanks for letting me know” after every user response.
- Only say thank you once at the end of the conversation when all required data is collected.
- For relevant questions, **offer clickable options** (e.g., [Under 1 year] [1–3 years] [Over 3 years]) to guide users and reduce confusion.

🧭 Main Flow: (for users who **have a business**)

1. Welcome & Permission
2. Type of Business
→ If user says they don’t have a business, **immediately switch to the 'Alternative Flow'** below.

3. Business Age  → e.g., [Under 6 months] [6–12 months] [1–3 years] [Over 3 years]
→ If the business is under 6 months, ask these 3 extra questions **one by one**, using buttons to guide the user:

  1. "What motivated you to start this business?"
   → e.g., [To earn income] [Passion project] [Saw a market need] [Family business] [Other]

  2. "Have you run any similar businesses or had related experience before?"
   → e.g., [Yes, similar business] [Related job experience] [No experience]

  3. "How stable is your current cash flow so far?"
   → e.g., [Very stable] [Somewhat stable] [Unstable]

These help us better understand the potential of the business despite its short operating history.

4. Monthly Income  → e.g., [Under $10] [$10–$30] [Over $30]
5. Existing Loans  → e.g., [Yes] [No]
6. Loan Purpose  → e.g., [Buy supplies] [Renovate space] [Marketing]
7. Desired Loan Amount → e.g., [$20] [$30] [$40] [$50]
8. Repayment Preference → e.g., [Daily] [Weekly] [Monthly]
9. Team Size → e.g., [Just me] [2–5] [6–10] [10+]
10. Selfie (Required)
11. Bonus Options (Optional)

🧭 Alternative Flow: (for users with **no business yet**)

1. Business Plan
2. Personal Income → e.g., [Under $10] [$10–$30] [Over $30]
3. Financial Stability
4. Guarantees  → e.g., [Personal savings] [Family support] [Other assets]
5. Experience  → e.g., [None] [Some experience] [Experienced]
6. Market Research
7. Loan Amount → e.g., [$20] [$30] [$40] [$50]
8. Repayment Plan → e.g., [Daily] [Weekly] [Monthly]
9. Documentation (Optional)

💬 Closing:
When all data is collected, respond with:
"Thank you! We have all the information we need."
""")


EXTRACTION_TEMPLATE = PromptTemplate.from_template("""
You are a highly skilled loan evaluation assistant.

Your job is to:
1. Extract the user's key details from a prior conversation (as structured fields).
2. Assign a **creditworthiness score from 0 to 10** to each relevant field.
3. Provide a **reason for each score**, explaining the logic clearly.
4. Generate an **overall credit score (0–10)** and a short **overall explanation**.

🧾 Output Format:
Return a `UserProfile` Pydantic object with all extracted fields.

Then return a separate JSON block like this:
{{
  "overall_score": 7.5,
  "overall_reason": "The user has a consistent income and no existing loans, but the business is under 1 year old.",
  "field_scores": [
    {{
      "field": "income",
      "score": 8,
      "reason": "The user's income is over $30/month, indicating strong repayment ability."
    }},
    {{
      "field": "existing_loans",
      "score": 10,
      "reason": "User has no existing loans, which lowers risk."
    }},
    {{
      "field": "business_age",
      "score": 5,
      "reason": "Business is under 1 year old, which is considered high-risk for lending."
    }}
    // ... include other relevant fields
  ]
}}

🎯 Scoring Guidelines:
- Scores range from 0 (very risky) to 10 (very safe/creditworthy).
- Be fair, balanced, and give both positive and negative reasons.
- Use domain-relevant logic (e.g., existing loans, income, repayment plan, business age, etc.).
- Do NOT hallucinate data; if unsure about a field, omit that score.

📝 Sample fields to evaluate (if available):
- income
- business_age
- existing_loans
- repayment_plan
- loan_amount
- guarantees
- experience
- financial_stability

⚠️ Never guess values not mentioned in the conversation.

📉 If the overall_score is 5.0 or lower:
- Add an "advice" field with 2–3 suggestions depending on their answers, so the user could consider to improve their creditworthiness.
- Focus on constructive, actionable tips (e.g., improving income consistency, building business history, offering collateral).

📦 Example addition:
"advice": [
  "Build 3–6 months of consistent income records.",
  "Try to repay existing debts before applying.",
  "Add a personal guarantee or family support to reduce perceived risk."
]

Begin now.
""")

def initialize_state(user_id: str):
    return {
        "messages": [SystemMessage(content=COLLECTION_TEMPLATE.format())],
        "collected_data": [],
        "is_finished": False
    }


class LLMResponse(BaseModel):
    is_finished: bool = Field(description="Indicates if data collection is complete")
    ai_message: str = Field(description="Message to display to the user")


def chatbot(state: State):
    messages = state["messages"]
    llm_with_output = llm.with_structured_output(LLMResponse)
    structured_response = llm_with_output.invoke(messages)

    if len(messages) > 1 and isinstance(messages[-1], HumanMessage):
        state["collected_data"].append({
            "user": messages[-1].content,
            "ai": structured_response.ai_message
        })

    state["messages"].append(AIMessage(content=structured_response.ai_message))
    state["is_finished"] = structured_response.is_finished

    return state


def end_state(state: State):
    return END if state.get("is_finished", False) else NODE1

graph_builder.add_node(NODE1, chatbot)
graph_builder.add_edge(START, NODE1)
graph_builder.add_edge(NODE1, END)
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)


### main_extraction

In [ ]:
# ==== UserProfile and Scoring ====
class UserProfile(BaseModel):
    has_business: bool = Field(..., description="Whether the user has an existing business")

    # Main Flow (for users who have a business)
    business_type: Optional[str] = Field(None, description="Type of business")
    business_age: Optional[str] = Field(None, description="Age of the business")

    # Extra 3 questions for under 6 months business
    motivation: Optional[str] = Field(None, description="Motivation behind starting the business")
    past_experience: Optional[str] = Field(None, description="Whether the user has had similar business experience before")
    cashflow_stability: Optional[str] = Field(None, description="How stable the current cash flow is")

    income: Optional[str] = Field(None, description="Monthly income from the business")
    existing_loans: Optional[str] = Field(None, description="Information about existing loans")
    loan_purpose: Optional[str] = Field(None, description="Purpose of the requested loan")
    loan_amount: Optional[str] = Field(None, description="Requested loan amount in USD")
    repayment_plan: Optional[str] = Field(None, description="How the user prefers to repay")
    team_size: Optional[str] = Field(None, description="Size of the user's business team")
    selfie_uploaded: Optional[bool] = Field(None, description="Whether the user uploaded a required selfie")
    bonus_options: Optional[str] = Field(None, description="Any additional optional uploads or information")

    # Alternative Flow (for users who don't have a business)
    business_plan: Optional[str] = Field(None, description="User's business idea or plan")
    personal_income: Optional[str] = Field(None, description="User's current income from other sources")
    financial_stability: Optional[str] = Field(None, description="Explanation of user's financial stability")
    guarantees: Optional[str] = Field(None, description="Any guarantees or collateral offered")
    experience: Optional[str] = Field(None, description="Relevant experience for the business idea")
    market_research: Optional[str] = Field(None, description="Evidence or explanation of market understanding")
    desired_loan_amount: Optional[str] = Field(None, description="Requested loan amount in USD")
    payment_plan: Optional[str] = Field(None, description="How the user prefers to repay")
    documentation_uploaded: Optional[bool] = Field(None, description="Whether the user uploaded any optional documents")


class FieldScore(BaseModel):
    field: str = Field(..., description="Name of the user data field that is being evaluated (e.g., 'income', 'business_age')")
    score: float = Field(..., description="Score assigned to this field, typically between 0 (poor) and 10 (excellent)")
    reason: str = Field(..., description="Short explanation justifying the score given for this field based on user's answer")

class ScoringSummary(BaseModel):
    overall_score: float = Field(..., description="Total creditworthiness score from 0 to 10 based on all responses")
    overall_reason: str = Field(..., description="Summary reason explaining why the user received this overall score")
    field_scores: List[FieldScore] = Field(..., description="Detailed list of scores and reasons for each evaluated field")
    advice: Optional[List[str]] = Field(..., description="Optional advice for improving creditworthiness if score is low")


# ==== Extraction Logic ====
def extract_user_data(messages: List[str]) -> UserProfile:
    system_message = [SystemMessage(content=EXTRACTION_TEMPLATE.format())]
    chat_history = [HumanMessage(content=f"{role}: {msg}") for role, msg in messages]

    profile = llm.with_structured_output(UserProfile).invoke(system_message + chat_history)
    return profile

def extract_user_scores(messages: List[str]) -> ScoringSummary:
    system_message = [SystemMessage(content=EXTRACTION_TEMPLATE.format())]
    chat_history = [HumanMessage(content=f"{role}: {msg}") for role, msg in messages]

    summary = llm.with_structured_output(ScoringSummary).invoke(system_message + chat_history)
    return summary


# ==== Run Loop ====
user_data_store={}

def invoke_graph_updates(user_input: str, user_id: str):
    config = {"configurable": {"thread_id": user_id}}

    if user_id not in user_data_store:
        user_data_store[user_id] = initialize_state(user_id)

    user_data_store[user_id]["messages"].append(HumanMessage(content=user_input))
    result = graph.invoke(user_data_store[user_id], config)
    user_data_store[user_id] = result

    response_msg = result["messages"][-1] if "messages" in result else None
    if isinstance(response_msg, AIMessage):
        print("Assistant:", response_msg.content)

    if result.get("is_finished"):
        print(f"\nConversation complete for user: {user_id}")
        chat_log = [("User", entry["user"]) for entry in result["collected_data"] if "user" in entry] + \
                   [("AI", entry["ai"]) for entry in result["collected_data"] if "ai" in entry]

        print("\nExtracted User Profile:")
        extracted = extract_user_data(chat_log)
        print(extracted.model_dump())

        print("\nCreditworthiness Scoring Summary:")
        score_summary = extract_user_scores(chat_log)
        print(score_summary.model_dump())

        return True

    return False


while True:    # interactive loop for multiple users
    user_id = input("Enter user ID (or 'quit' to exit): ")
    if user_id.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    while True:
      user_input= input(f"User ({user_id}): ")
      if user_input.lower() in ["quit", "exit", "q"]:
        break

      finished = invoke_graph_updates(user_input, user_id)
      if finished:
        break



Enter user ID (or 'quit' to exit): Pan
User (Pan): Hi
Assistant: Welcome! I'm here to help you apply for a small business loan. Do you already have a business?
User (Pan): yes, i have a business
Assistant: Great! To help with your loan application, can you please tell me what type of business you have? (e.g., retail, service, online, etc.)
User (Pan): AI integrated software company
Assistant: Thanks for sharing! How long has your business been operating? Please choose one of the following options: [Under 6 months] [6–12 months] [1–3 years] [Over 3 years]
User (Pan): under 6 months
Assistant: Since your business is under 6 months old, I'd like to ask you a few more questions to better understand your situation.

First, what motivated you to start this business? You can choose from the options below:

[To earn income] [Passion project] [Saw a market need] [Family business] [Other]
User (Pan): to earn income
Assistant: Next, have you run any similar businesses or had related experience be

KeyboardInterrupt: 

### extraction_method2

In [ ]:
user_data_store={}

class UserProfile(BaseModel):
    has_business: bool = Field(..., description="Whether the user has an existing business")

    # Main Flow (for users who have a business)
    business_type: Optional[str] = Field(None, description="Type of business")
    business_age: Optional[str] = Field(None, description="Age of the business")
    income: Optional[str] = Field(None, description="Monthly income from the business")
    existing_loans: Optional[str] = Field(None, description="Information about existing loans")
    loan_purpose: Optional[str] = Field(None, description="Purpose of the requested loan")
    loan_amount: Optional[str] = Field(None, description="Requested loan amount in USD")
    repayment_plan: Optional[str] = Field(None, description="How the user prefers to repay")
    team_size: Optional[str] = Field(None, description="Size of the user's business team")
    selfie_uploaded: Optional[bool] = Field(None, description="Whether the user uploaded a required selfie")
    bonus_options: Optional[str] = Field(None, description="Any additional optional uploads or information")

    # Alternative Flow (for users who don't have a business)
    business_plan: Optional[str] = Field(None, description="User's business idea or plan")
    personal_income: Optional[str] = Field(None, description="User's current income from other sources")
    financial_stability: Optional[str] = Field(None, description="Explanation of user's financial stability")
    guarantees: Optional[str] = Field(None, description="Any guarantees or collateral offered")
    experience: Optional[str] = Field(None, description="Relevant experience for the business idea")
    market_research: Optional[str] = Field(None, description="Evidence or explanation of market understanding")
    desired_loan_amount: Optional[str] = Field(None, description="Requested loan amount in USD")
    payment_plan: Optional[str] = Field(None, description="How the user prefers to repay")
    documentation_uploaded: Optional[bool] = Field(None, description="Whether the user uploaded any optional documents")


FIELD_MAPPING = {
    "what kind of business": "business_type",
    "how long have you been running": "business_age",
    "average monthly income": "income",
    "existing loans": "existing_loans",
    "purpose of the loan": "loan_purpose",
    "how much are you looking to borrow": "loan_amount",
    "repay the loan": "repayment_plan",
    "team size": "team_size",
    "upload a selfie": "selfie_uploaded",
    "bonus": "bonus_options",

    # Alternative Flow
    "business idea": "business_plan",
    "personal income": "personal_income",
    "financial stability": "financial_stability",
    "guarantees": "guarantees",
    "experience": "experience",
    "market understanding": "market_research",
    "how much are you looking to borrow": "desired_loan_amount",
    "repay your loan": "payment_plan",
    "upload any documents": "documentation_uploaded"
}

#extract user's data
def extract_user_data(collected_data: List[dict]) -> UserProfile:
    profile_data = {}
    last_ai_msg = ""

    for turn in collected_data:
        user_msg = turn.get("user", "")
        ai_msg = turn.get("ai", "")

        last_ai_msg = ai_msg.lower()

        for keyword, field in FIELD_MAPPING.items():
            if keyword in last_ai_msg:
                if field in ["selfie_uploaded", "documentation_uploaded"]:
                    profile_data[field] = "yes" in user_msg.lower() or "uploaded" in user_msg.lower()
                elif field == "existing_loans":
                    profile_data[field] = "yes" if "yes" in user_msg.lower() else "no"
                else:
                    profile_data[field] = user_msg
                break

        # Detect has_business early
        if any(x in last_ai_msg for x in ["do you currently have a business", "do you already have a business"]):
            if "yes" in user_msg.lower():
                profile_data["has_business"] = True
            elif "no" in user_msg.lower() or "planning to start" in user_msg.lower():
                profile_data["has_business"] = False

    if "has_business" not in profile_data:
        raise ValueError("Missing required field: 'has_business'. Conversation might be incomplete.")

    return UserProfile(**profile_data)


def invoke_graph_updates(user_input: str, user_id: str):
    config = {"configurable": {"thread_id": user_id}}

    if user_id not in user_data_store:
      user_data_store[user_id] = initialize_state(user_id)

    user_data_store[user_id]["messages"].append(HumanMessage(content=user_input))

    result = graph.invoke(user_data_store[user_id], config)
    user_data_store[user_id] = result

    response_msg = result["messages"][-1] if "messages" in result else None

    if isinstance(response_msg, AIMessage):          # response if it's AI msg
      print("Assitant: ", response_msg.content)

    # print("is finished (graph):", result.get('is_finished'))

    if result.get('is_finished'):
        print(f"\nConversation complete for user: {user_id}")
        print("Collected data:")
        for entry in result["collected_data"]:
            print(f"\nUser: {entry['user']}")
            print(f"AI: {entry['ai']}")

        print("\nExtracted user profile: ")
        extracted = extract_user_data(result["collected_data"])
        print(extracted.model_dump())
        return True

    return False


while True:    # interactive loop for multiple users
    user_id = input("Enter user ID (or 'quit' to exit): ")
    if user_id.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    while True:
      user_input= input(f"User ({user_id}): ")
      if user_input.lower() in ["quit", "exit", "q"]:
        break

      finished = invoke_graph_updates(user_input, user_id)
      if finished:
        break



Enter user ID (or 'quit' to exit): 87
User (87): hi
Assitant:  Hello! Are you looking to apply for a small business loan today?
User (87): i am
Assitant:  Great! Do you currently have a business, or are you planning to start one? This will help me guide you through the appropriate steps.
User (87): plannig to start one
Assitant:  Got it. Let's start with your business plan. Could you briefly describe your business idea? This helps lenders understand your vision and how you plan to use the loan.
User (87): i want to open a small grocery shop for food and drinks
Assitant:  That sounds like a great idea! Next, could you tell me about your personal income? This helps in assessing your current financial situation. Choose one of the ranges below:

- [Under $10]
- [$10–$30]
- [Over $30]
User (87): $30
Assitant:  Got it. Now, let's discuss financial stability. This involves understanding any existing financial resources you might have. Do you have any guarantees or sources of support you can r

ValueError: Missing required field: 'has_business'. Conversation might be incomplete.

In [ ]:
\